# Data Extraction from *geo-sorted ohsome contributions* 
<div class="alert alert-block alert-info">
<p>Set the query params</p>
<p>Set the connection params to Iceberg Rest Catalog and Minio S3 Storage</p>
    
<p>Prepare the data in 3 steps:</p>
<p>Do an iceberg table scan with a pre-filter</p>
<p>Fine filter the data in a Dataframe after download</p>
<p>Export results into geopackage file.</p>
</div>

# Getting started
Set connection params.

In [2]:
import os

s3_user = os.environ["S3_ACCESS_KEY_ID"]  # add your user here
s3_password = os.environ["S3_SECRET_ACCESS_KEY"]  # add your password here

In [3]:
from pyiceberg.catalog.rest import RestCatalog

catalog = RestCatalog(
    name="default",
    **{
        "uri": "https://sotm2024.iceberg.ohsome.org",
        "s3.endpoint": "https://sotm2024.minio.heigit.org",
        "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
        "s3.access-key-id": s3_user,
        "s3.secret-access-key": s3_password,
        "s3.region": "eu-central-1"
    }
)

# iceberg table
namespace = 'geo_sort'
tablename = 'contributions'
icebergtable = catalog.load_table((namespace, tablename))

Configure DuckDB.

In [4]:
import duckdb

con = duckdb.connect(
    config={
        'threads': 8,
        'max_memory': '8GB'
    }
)
con.install_extension("spatial")
con.load_extension("spatial")

# Iceberg to DuckDB
In this step we can already filter all OSM contributions by four major factors. We will perform more detailed filtering (e.g. for OSM tags values) later:
* **status** (e.g. latest, historic or deleted OSM features)
* **location** (using the bounding box coordinates of each OSM feature)
* **geometry type** (e.g. for Polygons, Linestrings or Points)
* **time** (e.g. the edit timestamp of each OSM contribution)

In [5]:
# Define status filter
status = 'latest'

# Define location filter
bboxes = {
    'heidelberg': (8.629761, 49.379556, 8.742371, 49.437890),
    'nairobi': (36.650938, -1.444471, 37.103887, -1.163522),
    'mannheim': (8.41416, 49.410362, 8.58999, 49.590489), 
    'berlin': (13.088345, 52.338271, 13.761161, 52.675509)
}
xmin, ymin, xmax, ymax = bboxes['heidelberg']

# Define geometry type filter
geometry_type = 'Polygon'

# Define time filter (optional)
min_timestamp = '2024-01-01T00:00:00'
max_timestamp = '2024-06-01T00:00:00'

Furthermore, we define which attributes / columns this download should contain. Check out the [dataset description page](./README.md) to get an overview on all available columns.

Usually you rarely want to extract all available columns as this would reduce speed of the data download. Here we are going to download the following information:

In [6]:
selected_fields = [
    "user_id",
    "osm_id",
    "osm_version",
    "valid_from",
    "tags",
    "geometry" 
]

<div class="alert alert-block alert-info">
<p><b>Info:</b> Download speed matters only in this step.</p>
<p>This is the only step in which we will download data from the server to our client (e.g. your laptop or jupyter notebook server). Internet connection and overall data size are the most common potential bottlenecks for this part of the analysis.</p>
<p>We have optimized the structure for all tables in the <i>geo_sort</i> namespace to filter for status, geometry_type and location.</p>
</div>

In [7]:
import time

start_time = time.time()

icebergtable.scan(
    row_filter= (
        f"status = '{status}'"
        f"and geometry_type = '{geometry_type}'"
        #f"and (bbox.xmax >= {xmin} and bbox.xmin <= {xmax})"
        #f"and (bbox.ymax >= {ymin} and bbox.ymin <= {ymax})"
        # optional timestamp filter
        # f"and valid_from >= '{min_timestamp}'"
        # f"and valid_from < '{max_timestamp}'"
    ),
    selected_fields=selected_fields,
    # optional: limit the number of features downloadd    
    limit=50000
).to_duckdb('osm_data',connection=con)

download_time = round(time.time() - start_time, 3)
print(f"download took {download_time} sec.")

download took 29.274 sec.


# DuckDB to GeoPackage

Show the structure of the data we have just downloaded.

In [8]:
query = """
DESCRIBE
FROM osm_data;
"""
con.sql(query)

┌─────────────┬───────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │      column_type      │  null   │   key   │ default │  extra  │
│   varchar   │        varchar        │ varchar │ varchar │ varchar │ varchar │
├─────────────┼───────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ user_id     │ INTEGER               │ YES     │ NULL    │ NULL    │ NULL    │
│ valid_from  │ TIMESTAMP             │ YES     │ NULL    │ NULL    │ NULL    │
│ osm_id      │ VARCHAR               │ YES     │ NULL    │ NULL    │ NULL    │
│ osm_version │ INTEGER               │ YES     │ NULL    │ NULL    │ NULL    │
│ tags        │ MAP(VARCHAR, VARCHAR) │ YES     │ NULL    │ NULL    │ NULL    │
│ geometry    │ VARCHAR               │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴───────────────────────┴─────────┴─────────┴─────────┴─────────┘

Inspect a few features.

In [9]:
query = """
SELECT *
FROM osm_data
LIMIT 5;
"""
con.sql(query)

┌─────────┬─────────────────────┬──────────────────┬─────────────┬──────────────────────┬──────────────────────────────┐
│ user_id │     valid_from      │      osm_id      │ osm_version │         tags         │           geometry           │
│  int32  │      timestamp      │     varchar      │    int32    │ map(varchar, varch…  │           varchar            │
├─────────┼─────────────────────┼──────────────────┼─────────────┼──────────────────────┼──────────────────────────────┤
│  274857 │ 2023-08-11 14:30:40 │ relation/8230041 │           2 │ {name=绿林镇, type…  │ POLYGON ((113.165726499999…  │
│ 9560399 │ 2024-07-25 17:24:19 │ relation/3216399 │          12 │ {name=安陆市, type…  │ POLYGON ((113.4821176 31.1…  │
│ 9560399 │ 2024-07-25 17:24:19 │ relation/3216402 │          10 │ {name=孝昌县, type…  │ POLYGON ((113.923309499999…  │
│  274857 │ 2023-08-11 14:30:40 │ relation/8230019 │           4 │ {name=罗店镇, type…  │ POLYGON ((113.4821176 31.1…  │
│ 9560399 │ 2023-07-03 16:12:22 │ relation/8

Count the number of features in the table when applying a more detailed tag filter.

Furthermore, apply detailed geometry filter for Heidelberg boundary.

In [19]:
query = """
SELECT count(*)
FROM
    osm_data,
    st_read('./data/Heidelberg.geojson') as heidelberg
WHERE 1=1
    -- filter for all boundaries in OSM --> boundary=*
    and list_contains(map_keys(tags), 'boundary')
    -- intersect osm data with Heidelberg boundary
    and ST_Intersects(st_GeomFromText(osm_data.geometry), heidelberg.geom)
"""
con.sql(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│            6 │
└──────────────┘

Export as GeoPackage via GeoPandas.

In [20]:
import geopandas as gpd

query = f"""
    SELECT *
    FROM
        osm_data,
        st_read('./data/Heidelberg.geojson') as heidelberg
    WHERE 1=1
        -- filter for all boundaries in OSM --> boundary=*
        and list_contains(map_keys(tags), 'boundary')
        -- intersect osm data with Heidelberg boundary
        and ST_Intersects(st_GeomFromText(osm_data.geometry), heidelberg.geom)
"""
df = con.sql(query).df()

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.GeoSeries.from_wkt(df['geometry'])
).set_crs('epsg:4326')

output_filename = "heidelberg_osm_data.gpkg"
gdf.to_file(output_filename, driver='GPKG')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Work with the data in QGIS
Add your geopackage file in QGIS, e.g. via drag-and-drop or through file manager.